## This lab explains simple example to build AI Agent.

In [2]:
# @title Authentication

%pip install --upgrade --quiet google-cloud-aiplatform

#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 50.3 MB/s eta 0:00:00


In [3]:
# @title Agent initialization.

from IPython.display import display, Markdown

import vertexai
import vertexai.generative_models as generative_models
from vertexai.preview.generative_models import grounding

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

In [1]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을  참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 제품 추천 : http://sec_vd.com/제품검색
  2. 대리점 추천 : http://sec_vd.com/대리점추천
  3. 번역테스트 : http://sec_vd.com/번역테스트
  4. 대리점 수리접수 : http://sec_vd.com/대리점수리접수
  5. 기타 : http://sec_vd.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def get_prompts(chat_history, query, location):

  prompt_1 =f"""
          당신은 삼성 제품 추천을 위한 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 삼성 제품 추천에 대해서 제품명 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 추천을 하는 지역위치는 최신 대화의 지역을 우선으로 추천해주세요.
          3. 질문 또는 대화내용에서 특정 위치를 알려주지 않으면 기본값으로 {location} 지역을 기반으로 답해주세요.
          4. 사전 훈련된 지식에서 질문하는 지역에서 자주 언급되는 제품을 이용해서 추천해주시고, 이유를 알려주세요.
          5. 자세한 답변을 위해서 추가적으로 사용자의 관심사항(색상,크기,기능등을) 요청해주세요.
          6. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_2 = f"""당신은 삼성 가전 제품 사용자를 위해 번역해주는 AI 어시스턴트 입니다. 아래 내용에 맞게 번역해주세요.

          1. 번역 대상 언어 선택은 가장 최근 대화를 참고해서 선택하고, 없다면 번역대상 언어 지정을 사용자에게 요청해주세요.
          2. 번역을 해야할 부분은 요청하는 문맥에 맞게 선택해서 번역해주세요.
          3. 특별하게 번역해야 할 부분을 제시하지 않으면 전체 문장을 번역해주세요.
          4. 답변은 최대한 간략하게 번역된 부분만 표현하고 설명하지 말아주세요.

          <번역을 위한 질문> : {query}

  """

  prompt_3 = f"""당신은 삼성 제품 사용자를 위해 삼성 대리점 위치 추천해주는 AI Assistant입니다.

          1. 질문에 맞는 삼성제품을 구매할 수 있는 대리점 명을 3~4개 정도 추천해주세요.
          2. 해당 대리점의 주소를 알려주고 왜 추천하는지 알려주세요.
          3. 전체적인 답변은 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_4 = f"""당신은 삼성제품 사용자를 위해 대리점 수리 접수를 도와주는 AI Assistant입니다.

          1. 수리접수를 위한 수리점 위치는 최신 대화의 지역을 우선으로 구글에서 검색해서 추천해주세요.
          2. 질문 또는 대화내용에서 특정 위치를 알려주지 않으면 기본값으로 {location} 지역을 기반으로 답해주세요.
          3. 수리점의 정확한 주소와 전화번호를 같이 보여주세요.
          4. 만일 정확한 지역정보 또는 수리점 위치를 모를 때는 해당 위치정보를 요청해주세요.
          5. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  prompt_5 = f"""당신은 삼성제품 사용자의 질문에 답변을 해주는 AI Assistant입니다.
          사용자의 다양한 질문에 대해서 검색 결과를 활용해서 답변 해주세요.
          가급적 답변은 10줄 이내로 짧고 간략하게 답해주세요.
          <질문> : {query}

  """

  classified = query_classification(chat_history, query)

  task = eval(classified)['classification']['class']

  if task == '제품 추천':
    return prompt_1, True
  elif task == '번역테스트':
    return prompt_2, False
  elif task == '대리점 추천':
    return prompt_3, True
  elif task == '대리점 수리접수':
    return prompt_4, True
  elif task == '기타':
    return prompt_5, True
  else:
    return prompt_5, True


In [6]:
# @title AI Agent

chat = model.start_chat()

location = "수원시"
search = True

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  prompt, search = get_prompts(chat_history, query, location)

  response = interactive_chat(chat, prompt, search)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 최근 스릴러 영화 추천해주세요.


AI Agent : 최근 스릴러 영화 추천이요? 수원시에서 인기있는 스릴러 영화는 "콘크리트 유토피아"와 "콜" 이에요. 

"콘크리트 유토피아"는 재난 상황 속에서 인간의 어두운 심리를 잘 보여주는 영화고, "콜"은 시간을 초월한 전화 연결을 소재로 긴장감 넘치는 스토리를 선사하는 영화랍니다. 

어떤 종류의 스릴러 영화에 관심있으세요? 좀 더 자세히 알려주시면 취향에 맞는 영화를 추천해드릴 수 있어요! 예를 들어, 어둡고 잔혹한 스릴러를 좋아하시는지, 아니면 반전이 있는 스릴러를 좋아하시는지 말씀해주세요. 😊 


------------------------------------ 


KeyboardInterrupt: Interrupted by user